# Train the Final Models for Production

## Initial Setup

In [ ]:
!pip install -U ultralytics wandb

In [ ]:
import os
import yaml

from ultralytics import YOLO

import wandb
from wandb.integration.ultralytics import add_wandb_callback

## Initialize the Experiment

In [ ]:
sweep_run = "reviewco/object-detection-bdd/2yvwiulh"

wandb.init(
    project="object-detection-bdd",
    config=wandb.Api().run(sweep_run).config, # Set the configs to the the configs of the best candidate from your sweep
    job_type="final/train",
)

config = wandb.config
config.epochs = 30

## Fetch and Preprocess the Dataset

In [ ]:
# Fetch the bdd-100k dataset hosted as a W&B dataset artifact
artifact = wandb.use_artifact(
    "reviewco/object-detection-bdd/bdd100k-ultralytics-format:latest", type="dataset"
)
artifact_dir = artifact.download()

# Preprocess the data.yaml file in the dataset
metadata_file = os.path.join(artifact_dir, "data.yaml")
with open(metadata_file, "r") as yaml_file:
    metadata = yaml.safe_load(yaml_file)
metadata["path"] = os.path.join(
    os.path.dirname(os.path.realpath(__file__)), artifact_dir
)
with open(metadata_file, "w") as yaml_file:
    yaml.dump(metadata, yaml_file)

## Training and Validation

In [ ]:
model = YOLO(f"{config.model}.pt")

add_wandb_callback(model, enable_model_checkpointing=True, max_validation_batches=4)

In [ ]:
# Train with all the hyperparameters set to the desired values
model.train(
    data=metadata_file,
    epochs=config.epochs,
    batch=4,
    imgsz=config.imgsz,
    lr0=config.lr0,
    lrf=config.lrf,
    momentum=config.momentum,
    weight_decay=config.weight_decay,
    warmup_epochs=config.warmup_epochs,
    warmup_momentum=config.warmup_momentum,
    box=config.box,
    cls=config.cls,
    hsv_h=config.hsv_h,
    hsv_s=config.hsv_s,
    hsv_v=config.hsv_v,
    degrees=config.degrees,
    translate=config.translate,
    scale=config.scale,
    shear=config.shear,
    perspective=config.perspective,
    flipud=config.flipud,
    fliplr=config.fliplr,
    mosaic=config.mosaic,
    mixup=config.mixup,
    copy_paste=config.copy_paste,
)

# Validate the model
model.val()

# Finish the experiment
wandb.finish()